In [1]:
import openai
import json
import hashlib
import random
import time
from typing import List, Dict, Set, Tuple
import re
import os
from datetime import datetime
from collections import defaultdict
import pandas as pd
from IPython.display import display, HTML, clear_output

In [ ]:
class ExtendedTurkishCDatasetGenerator:
    def __init__(self, api_key: str):
        self.client = openai.OpenAI(api_key=api_key)
        self.generated_questions = set()
        self.dataset = []
        self.problem_types = self._define_problem_types()
        self.c_concepts = self._define_c_concepts()
        self.question_templates = self._define_question_templates()

    def _define_problem_types(self) -> Dict[str, Dict]:
        """Problem türlerini ve bunlar için prompt şablonlarını tanımla"""
        return {
            "code_completion": {
                "description": "Eksik kod tamamlama",
                "system_prompt": "Sen bir C programlama uzmanısın. Kullanıcının verdiği eksik kodu tamamla ve açıkla.",
                "difficulty_levels": ["basic", "intermediate", "advanced"]
            },
            "debugging": {
                "description": "Hata ayıklama ve düzeltme",
                "system_prompt": "Sen bir C debugging uzmanısın. Hataları bul, düzelt ve nedenlerini açıkla.",
                "difficulty_levels": ["basic", "intermediate", "advanced"]
            },
            "explanation": {
                "description": "Kod açıklama ve analiz",
                "system_prompt": "Sen bir C eğitmenisin. Kodları adım adım ve anlaşılır şekilde açıkla.",
                "difficulty_levels": ["basic", "intermediate", "advanced"]
            },
            "implementation": {
                "description": "Sıfırdan implementasyon",
                "system_prompt": "Sen bir C developer'ısın. İstenen algoritma/yapıyı sıfırdan kodla ve açıkla.",
                "difficulty_levels": ["basic", "intermediate", "advanced"]
            },
            "code_review": {
                "description": "Kod inceleme ve optimizasyon",
                "system_prompt": "Sen bir senior C developer'ısın. Kodları incele, optimize et ve iyileştirme önerileri sun.",
                "difficulty_levels": ["intermediate", "advanced"]
            }
        }

    def _define_c_concepts(self) -> Dict[str, Dict]:
        """C dilinin temel konularını tanımla"""
        return {
            "basic_syntax": {
                "description": "Temel C Sözdizimi",
                "topics": {
                    "variables_datatypes": "Değişkenler ve veri tipleri",
                    "operators": "Operatörler",
                    "control_structures": "Kontrol yapıları",
                    "functions": "Fonksiyonlar",
                    "preprocessor": "Preprocessor directiveleri"
                }
            },
            "arrays_strings": {
                "description": "Diziler ve Stringler",
                "topics": {
                    "arrays_1d": "Tek boyutlu diziler",
                    "arrays_2d": "Çok boyutlu diziler",
                    "string_basics": "String temelleri",
                    "string_functions": "String fonksiyonları",
                    "character_arrays": "Karakter dizileri"
                }
            },
            "pointers_memory": {
                "description": "Pointerlar ve Bellek",
                "topics": {
                    "pointer_basics": "Pointer temelleri",
                    "dynamic_memory": "Dinamik bellek yönetimi",
                    "pointer_arithmetic": "Pointer aritmetiği",
                    "function_pointers": "Function pointerlar",
                    "memory_leaks": "Bellek sızıntıları"
                }
            },
            "data_structures": {
                "description": "Veri Yapıları",
                "topics": {
                    "linked_lists": "Bağlı listeler",
                    "stacks_queues": "Stack ve Queue",
                    "trees": "Ağaç yapıları",
                    "sorting": "Sıralama algoritmaları",
                    "searching": "Arama algoritmaları"
                }
            },
            "file_operations": {
                "description": "Dosya İşlemleri",
                "topics": {
                    "file_basics": "Dosya okuma/yazma temelleri",
                    "binary_files": "Binary dosya işlemleri",
                    "text_processing": "Metin dosyası işleme",
                    "file_streams": "Dosya stream'leri"
                }
            },
            "advanced_topics": {
                "description": "İleri Düzey Konular",
                "topics": {
                    "structures": "Struct yapıları",
                    "unions": "Union kullanımı",
                    "bit_operations": "Bit işlemleri",
                    "recursion": "Recursion (özyinelemeli fonksiyonlar)",
                    "command_line": "Komut satırı argümanları"
                }
            }
        }

    def _define_question_templates(self) -> Dict[str, List[str]]:
        """Soru şablonları"""
        return {
            "code_completion": [
                "Aşağıdaki {topic} kodunu tamamlayın:\n{code}",
                "Bu {topic} kodundaki eksik bölümleri doldurun:\n{code}",
                "{topic} ile ilgili bu kodu tamamlayarak çalışır hale getirin:\n{code}"
            ],
            "debugging": [
                "Bu {topic} kodundaki hataları bulun ve düzeltin:\n{code}",
                "Aşağıdaki {topic} kodunda neler yanlış?\n{code}",
                "Bu {topic} kodunu çalıştırdığımda hata alıyorum, nedenini açıklayın:\n{code}"
            ],
            "explanation": [
                "Bu {topic} kodunun nasıl çalıştığını açıklayın:\n{code}",
                "Aşağıdaki {topic} kodunu satır satır açıklayın:\n{code}",
                "{topic} konusundaki bu kod örneğini detaylıca anlatın:\n{code}"
            ],
            "implementation": [
                "{topic} için {task} implementasyonu yazın",
                "C dilinde {topic} ile {task} nasıl yapılır?",
                "{topic} kullanarak {task} algoritmasını kodlayın"
            ],
            "code_review": [
                "Bu {topic} kodunu inceleyin ve iyileştirme önerileri sunun:\n{code}",
                "Aşağıdaki {topic} kodunun performansını nasıl artırabiliriz?\n{code}"
            ]
        }

    def _generate_bulk_questions(self, category: str, topic: str, problem_type: str,
                              difficulty_level: str, count: int = 5) -> List[Dict]:
        """Bulk soru üretimi"""

        topic_name = self.c_concepts[category]["topics"][topic]
        system_prompt = f"""{self.problem_types[problem_type]['system_prompt']}

        Zorluk seviyesi: {difficulty_level}
        Konu: {topic_name}

        Tüm cevapların Türkçe olmalı. Kod örnekleri verirken açıklamalar da ekle.
        Gerçekçi ve pratik örnekler üret. Her soru-cevap çifti benzersiz olmalı.

        Lütfen {count} farklı soru-cevap çifti üret. JSON formatında ver:

        {{"examples": [
          {{"question": "soru metni", "answer": "detaylı cevap"}},
          ...
        ]}}"""

        user_prompt = f"""Lütfen {topic_name} konusu ile ilgili {problem_type} türünde {count} farklı
        soru-cevap çifti üret. Zorluk seviyesi: {difficulty_level}.
        Her soru benzersiz olmalı ve gerçek programlama senaryolarını yansıtmalı."""

        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=5000,
                temperature=0.8
            )

            response_text = response.choices[0].message.content.strip()

            # JSON parse et
            try:
                json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
                if json_match:
                    json_str = json_match.group()
                    data = json.loads(json_str)

                    questions = []
                    for example in data.get("examples", []):
                        question = example.get("question", "").strip()
                        answer = example.get("answer", "").strip()

                        if question and answer and not self._is_duplicate(question):
                            self._add_question_hash(question)
                            questions.append({
                                "question": question,
                                "answer": answer,
                                "category": category,
                                "topic": topic,
                                "problem_type": problem_type,
                                "difficulty": difficulty_level,
                                "topic_name": topic_name,
                                "timestamp": datetime.now().isoformat()
                            })

                    return questions

            except json.JSONDecodeError:
                print(f"JSON parse hatası: {category}/{topic}")
                return []

        except Exception as e:
            print(f"API Hatası: {e}")
            time.sleep(2)  # Rate limiting için bekleme
            return []

        return []

    def _create_question_hash(self, question: str) -> str:
        """Soru için hash oluştur"""
        normalized = question.lower()
        cleaned = re.sub(r'[^\w\s]', '', normalized)
        cleaned = re.sub(r'\s+', ' ', cleaned).strip()
        return hashlib.md5(cleaned.encode('utf-8')).hexdigest()

    def _is_duplicate(self, question: str) -> bool:
        """Duplikasyon kontrolü"""
        question_hash = self._create_question_hash(question)
        return question_hash in self.generated_questions

    def _add_question_hash(self, question: str):
        """Soru hash'ini kaydet"""
        question_hash = self._create_question_hash(question)
        self.generated_questions.add(question_hash)

    def generate_large_dataset(self, target_size: int = 1000, batch_save_size: int = 1000):
        """1000 örnek için büyük dataset üretimi"""
        print(f"🚀 {target_size} Örnekli Dataset Üretiliyor...")
        print("=" * 60)

        # Her kategori/konu/tip/zorluk kombinasyonu için kaç soru üretileceğini hesapla
        total_combinations = 0
        for category in self.c_concepts:
            topics = len(self.c_concepts[category]["topics"])
            problem_types = len(self.problem_types)
            difficulties = 3  # basic, intermediate, advanced
            total_combinations += topics * problem_types * difficulties

        questions_per_combination = max(1, target_size // total_combinations)

        print(f"📊 Toplam kombinasyon: {total_combinations}")
        print(f"🎯 Kombinasyon başına soru: {questions_per_combination}")
        print("-" * 40)

        total_generated = 0
        successful_batches = 0
        failed_batches = 0

        for category_key, category_data in self.c_concepts.items():
            print(f"\n📚 Kategori: {category_data['description']}")

            for topic_key, topic_name in category_data["topics"].items():
                print(f"  🔸 Konu: {topic_name}")

                for problem_type in self.problem_types.keys():
                    # Her problem tipi için farklı zorluk seviyeleri
                    available_difficulties = self.problem_types[problem_type]["difficulty_levels"]

                    for difficulty in available_difficulties:
                        print(f"    ⚡ {problem_type} ({difficulty}) soruları üretiliyor...")

                        questions = self._generate_bulk_questions(
                            category_key, topic_key, problem_type, difficulty,
                            questions_per_combination
                        )

                        if questions:
                            self.dataset.extend(questions)
                            total_generated += len(questions)
                            successful_batches += 1
                            print(f"      ✅ {len(questions)} soru eklendi (Toplam: {total_generated})")

                            if total_generated % batch_save_size == 0:
                                print(f"\n💾 {total_generated} soruya ulaşıldı, ara kayıt yapılıyor...")
                                self.save_dataset_with_metadata(base_filename=f"turkish_c_dataset_batch_{total_generated}")
                        else:
                            failed_batches += 1
                            print(f"      ❌ Soru üretilemedi")

                        # Rate limiting - OpenAI API limitleri için
                        time.sleep(1)

                        # İlerleme durumu
                        if total_generated >= target_size:
                            print(f"\n🎉 Hedef boyuta ulaşıldı! ({target_size})")
                            break

                    if total_generated >= target_size:
                        break
                if total_generated >= target_size:
                    break
            if total_generated >= target_size:
                break

        print(f"\n" + "="*60)
        print(f"🎉 Dataset üretimi tamamlandı!")
        print(f"📊 İstatistikler:")
        print(f"   • Toplam soru: {total_generated}")
        print(f"   • Başarılı batch: {successful_batches}")
        print(f"   • Başarısız batch: {failed_batches}")
        print(f"   • Başarı oranı: {(successful_batches/(successful_batches+failed_batches)*100):.1f}%")

        return self.dataset

    def display_progress_statistics(self):
        """İlerleme istatistikleri"""
        if not self.dataset:
            print("❌ Henüz soru üretilmedi!")
            return

        df = pd.DataFrame(self.dataset)

        print("📈 DATASET İSTATİSTİKLERİ")
        print("=" * 40)
        print(f"📊 Toplam soru sayısı: {len(df)}")
        print(f"🏷️ Benzersiz kategori: {df['category'].nunique()}")
        print(f"📝 Benzersiz konu: {df['topic'].nunique()}")
        print(f"🎯 Problem türü sayısı: {df['problem_type'].nunique()}")

        print(f"\n📏 Uzunluk İstatistikleri:")
        print(f"   • Ortalama soru uzunluğu: {df['question'].str.len().mean():.0f} karakter")
        print(f"   • Ortalama cevap uzunluğu: {df['answer'].str.len().mean():.0f} karakter")

        print(f"\n🏆 En Çok Soru Olan Kategoriler:")
        category_counts = df['category'].value_counts().head(5)
        for cat, count in category_counts.items():
            cat_name = self.c_concepts[cat]['description']
            print(f"   • {cat_name}: {count} soru")

        print(f"\n🎯 Problem Türü Dağılımı:")
        problem_counts = df['problem_type'].value_counts()
        for prob_type, count in problem_counts.items():
            print(f"   • {prob_type}: {count} soru")

        print(f"\n⚡ Zorluk Seviyesi Dağılımı:")
        difficulty_counts = df['difficulty'].value_counts()
        for diff, count in difficulty_counts.items():
            print(f"   • {diff}: {count} soru")

    def save_dataset_with_metadata(self, base_filename: str = "turkish_c_dataset_1000"):
        """Metadata ile birlikte dataset'i kaydet"""
        if not self.dataset:
            print("❌ Kaydedilecek dataset yok!")
            return

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Ana dataset
        main_filename = f"{base_filename}_{timestamp}.json"
        with open(main_filename, 'w', encoding='utf-8') as f:
            json.dump(self.dataset, f, ensure_ascii=False, indent=2)

        # Fine-tuning formatı
        jsonl_filename = f"{base_filename}_fine_tuning_{timestamp}.jsonl"
        self.create_fine_tuning_format(jsonl_filename)

        # CSV formatı (analiz için)
        csv_filename = f"{base_filename}_analysis_{timestamp}.csv"
        df = pd.DataFrame(self.dataset)
        df.to_csv(csv_filename, index=False, encoding='utf-8')

        # Metadata
        metadata = {
            "creation_date": datetime.now().isoformat(),
            "total_questions": len(self.dataset),
            "categories": list(self.c_concepts.keys()),
            "problem_types": list(self.problem_types.keys()),
            "statistics": {
                "avg_question_length": df['question'].str.len().mean(),
                "avg_answer_length": df['answer'].str.len().mean(),
                "category_distribution": df['category'].value_counts().to_dict(),
                "problem_type_distribution": df['problem_type'].value_counts().to_dict(),
                "difficulty_distribution": df['difficulty'].value_counts().to_dict()
            }
        }

        metadata_filename = f"{base_filename}_metadata_{timestamp}.json"
        with open(metadata_filename, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)

        print(f"💾 Dataset dosyaları kaydedildi:")
        print(f"   📄 Ana dataset: {main_filename}")
        print(f"   🎯 Fine-tuning: {jsonl_filename}")
        print(f"   📊 CSV analyz: {csv_filename}")
        print(f"   ℹ️  Metadata: {metadata_filename}")

    def create_fine_tuning_format(self, filename: str):
        """Fine-tuning formatında kaydet"""
        fine_tuning_data = []

        for item in self.dataset:
            fine_tuning_format = {
                "messages": [
                    {
                        "role": "system",
                        "content": "Sen Türkçe konuşan deneyimli bir C programcısısın. Kullanıcının sorularını detaylı ve öğretici şekilde cevapla. Kod örnekleri verirken açıklamalar da ekle."
                    },
                    {"role": "user", "content": item["question"]},
                    {"role": "assistant", "content": item["answer"]}
                ]
            }
            fine_tuning_data.append(fine_tuning_format)

        with open(filename, 'w', encoding='utf-8') as f:
            for item in fine_tuning_data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')

        return fine_tuning_data

def generate_X_examples(api_key: str, example_size: int):
    """1000 örnekli dataset üret"""
    print("🚀 1000 ÖRNEKLI TÜRKÇE C DATASET ÜRETME")
    print("=" * 50)

    # Generator oluştur
    generator = ExtendedTurkishCDatasetGenerator(api_key)

    # x örnek üret
    dataset = generator.generate_large_dataset(example_size, batch_save_size=1000)


    # İstatistikleri göster
    generator.display_progress_statistics()

    # Kaydet
    generator.save_dataset_with_metadata("turkish_c_dataset")

    print("\n✅ İşlem tamamlandı!")
    return generator

# Hızlı test için küçük örnek
def quick_test(api_key: str):
    """Hızlı test - 5 örnek"""
    generator = ExtendedTurkishCDatasetGenerator(api_key)
    dataset = generator.generate_large_dataset(5)
    generator.display_progress_statistics()
    generator.save_dataset_with_metadata("small_dataset")
    return generator

if __name__ == "__main__":
    # Kullanım örneği
    API_KEY ="your-OpenAI-api-key"

    # 1000 örnekli dataset üret
    generator = generate_X_examples(API_KEY, 5000)

🚀 1000 ÖRNEKLI TÜRKÇE C DATASET ÜRETME
🚀 5000 Örnekli Dataset Üretiliyor...
📊 Toplam kombinasyon: 435
🎯 Kombinasyon başına soru: 11
----------------------------------------

📚 Kategori: Temel C Sözdizimi
  🔸 Konu: Değişkenler ve veri tipleri
    ⚡ code_completion (basic) soruları üretiliyor...
      ✅ 10 soru eklendi (Toplam: 10)
    ⚡ code_completion (intermediate) soruları üretiliyor...
      ✅ 10 soru eklendi (Toplam: 20)
    ⚡ code_completion (advanced) soruları üretiliyor...
      ✅ 10 soru eklendi (Toplam: 30)
    ⚡ debugging (basic) soruları üretiliyor...
      ✅ 10 soru eklendi (Toplam: 40)
    ⚡ debugging (intermediate) soruları üretiliyor...
      ✅ 10 soru eklendi (Toplam: 50)
    ⚡ debugging (advanced) soruları üretiliyor...
JSON parse hatası: basic_syntax/variables_datatypes
      ❌ Soru üretilemedi
    ⚡ explanation (basic) soruları üretiliyor...
      ✅ 11 soru eklendi (Toplam: 61)
    ⚡ explanation (intermediate) soruları üretiliyor...
      ✅ 10 soru eklendi (Toplam: 7